# Implementation: Volumetric Rendering (NeRF Logic)

**Goal**: Ray Marching.

In [ ]:
import torch

# 1. Ray samples (Distance along the ray)
t_vals = torch.linspace(0, 1, 5) # 5 samples

# 2. Predictions from NeRF (Mock)
# Colors (RGB) and Density (Sigma)
colors = torch.tensor([[1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [0, 0, 0]]).float() 
densities = torch.tensor([0.0, 0.0, 0.9, 0.1, 0.0]) # Hit a green object at step 2

# 3. Alpha Composition
def render_ray(colors, densities, delta=0.2):
    # Alpha = 1 - e^(-sigma * delta)
    alpha = 1.0 - torch.exp(-densities * delta)
    
    # Transmittance (How much light gets through previous layers)
    # T_i = Product(1 - alpha_j) for j < i
    T = torch.cumprod(1.0 - alpha + 1e-10, dim=0)
    T = torch.cat([torch.ones(1), T[:-1]]) # Shift right
    
    # Final Color = Sum(T * alpha * color)
    weights = T * alpha
    final_color = (weights.unsqueeze(1) * colors).sum(dim=0)
    return final_color, weights

pixel_color, weights = render_ray(colors, densities)
print(f"Pixel Color: {pixel_color.numpy()} (Mostly Green)")
print(f"Weights: {weights.numpy()} (Peak at sample 2)")

## Conclusion
This differentiable rendering allows us to backpropagate from a 2D photo error to update the 3D density field.